In [1]:
from Okezone import Okezone
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [2]:
obj = Okezone()

In [3]:
end_date = date(2018, 8, 12)
start_date = date(2018, 2, 3)
init_page = 4

In [4]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, single_date.strftime("%Y/%m/%d"))
    init_page = 1

2018-02-22
page  1
https://index.okezone.com/bydate/index/2018/02/22/0/
memasukkan berita id  1863481
Insert berita  Mengaku Simpatisan PKI, Pemuda Bertato Dihajar Massa di Pancoran
masuk
memasukkan berita id  1863480
Insert berita  HiVi Tampil Memukau Selepas Pengumuman 5 Besar Miss Indonesia 2018
masuk
memasukkan berita id  1863478
Insert berita  Ini Dia 5 Besar Finalis Terbaik di Malam Puncak Miss Indonesia 2018
masuk
memasukkan berita id  1863367
Insert berita  Zarco Yakin Syahrin Akan Berkembang Pesat di MotoGP 2018
masuk
memasukkan berita id  1863475
Insert berita  Nia Ramadhani Tak Khawatir Suaminya Banyak Didekati Wanita Cantik
masuk
memasukkan berita id  1863477
Insert berita  Dinilai Berhasil Edukasi Masyarakat, Okezone.com Terima Penghargaan dari BNPB
masuk
memasukkan berita id  1863350
Insert berita  Ferdinand Puji Kemampuan Apik De Gea Usai Hadapi Sevilla
masuk
memasukkan berita id  1863467
Insert berita  Jelang Pernikahan, Meghan Markle Dikabarkan Selingkuhi Pangeran Harr

masuk
memasukkan berita id  1863405
Insert berita  Cari PNS Profesional, BKN Seleksi 2.500 ASN   
masuk
memasukkan berita id  1863404
Insert berita  Cegah Napi Kambuhan, Lapas Tangerang Edukasi Melalui Kegiatan Seni
masuk
memasukkan berita id  1863403
Insert berita  Tayang Perdana, <i>Meet Me After Sunset</i> Gelar Nobar di Depok
masuk
memasukkan berita id  1863402
Insert berita  Staf TU SMPN 7 Sarolangun Bawa Kabur Siswinya   
masuk
memasukkan berita id  1863401
Insert berita  Perindo Cianjur Siap <i>All Out</i> Menangkan Deddy Mizwar-Dedi Mulyadi
masuk
memasukkan berita id  1863400
Insert berita  Sulap Catwalk Jadi Ruang Operasi, Gucci Tampilkan Model Bermata Tiga hingga Membawa Iguana
masuk
page  6
https://index.okezone.com/bydate/index/2018/02/22/75/
memasukkan berita id  1863399
Insert berita  Sutradara Dipecat, Sekuel Meteor Garden Ditunda
masuk
memasukkan berita id  1863398
Insert berita  Sowan ke Sejumlah Kiai, Gus Ipul Didoakan Menang Pilgub Jatim
masuk
memasukkan berita id  1

masuk
memasukkan berita id  1863325
Insert berita  Mira Lesmana Harapkan Adanya Usaha Pemerintah dalam Menumbuhkan Film Anak
masuk
memasukkan berita id  1863324
Insert berita  Operasi Mata Tahap Kedua Novel Baswedan Akan Secepatnya Dilakukan
masuk
memasukkan berita id  1863323
Insert berita  Setnov Disebut Tunjuk Orang Jadi Anggota BPK untuk Muluskan Audit Proyek E-KTP
masuk
memasukkan berita id  1863322
Insert berita  Vakum 2 Tahun, So Ji Sub Siap <i>Comeback</i> Lewat <i>Terius Behind Me</i>
masuk
memasukkan berita id  1863316
Insert berita  Lepas Mahkota Malam Ini, Begini Pesan Achintya Nilsen untuk Miss Indonesia 2018
masuk
memasukkan berita id  1863320
Insert berita  Mira Lesmana Jelaskan Mengapa Film Anak Langka di Indonesia
masuk
memasukkan berita id  1863319
Insert berita  Jangan Pergi ke 5 Tempat Ini! Berbahaya
masuk
memasukkan berita id  1863317
Insert berita  Indonesia-Bulgaria Jajaki Kerjasama Pengembangan Energi Terbarukan
masuk
memasukkan berita id  1863315
Insert berita 

masuk
memasukkan berita id  1863241
Insert berita  Pemulihan Kesehatan di Asmat, Kemenkes Kirim Tim Medis dan Logistik
masuk
memasukkan berita id  1863239
Insert berita  Manajer Repsol Honda Kagum dengan Kemampuan Folger 
masuk
page  15
https://index.okezone.com/bydate/index/2018/02/22/210/
memasukkan berita id  1863238
Insert berita  Bawa Obat Golongan Psikotropika, Wisatawan Asal Inggris Terancam Hukuman 10 Tahun Penjara
masuk
memasukkan berita id  1863237
Insert berita    Desainer Asal Jerman Sembunyikan Narkoba di Celana Dalam   
masuk
memasukkan berita id  1863236
Insert berita  Baru Adhi Karya yang Ajukan Konsultan Pembangunan Infrastruktur <i>Elevated</i>   
masuk
memasukkan berita id  1863235
Insert berita  Indonesia Impor Daging Kerbau Beku dari India
masuk
memasukkan berita id  1863234
Insert berita  Sebelum Wamil, G-Dragon Resmikan Kafe di Pulau Jeju
masuk
memasukkan berita id  1863232
Insert berita  Jokowi Perkirakan Revitalisasi Sungai Citarum Selesai dalam Waktu 7 Tahun
m

masuk
memasukkan berita id  1863170
Insert berita  Guru Besar Diimbau untuk Tingkatkan Riset   
masuk
memasukkan berita id  1863169
Insert berita  Genjot Wisatawan, <i>Mlaku-Mlaku Nang</i> Tunjungan Kini Digelar Setiap Bulan
masuk
memasukkan berita id  1863168
Insert berita  Jurrasic World 3 Siap Tayang di Bioskop pada 2021
masuk
memasukkan berita id  1863167
Insert berita  Novel Belum Sembuh dan Akan Kembali Jalani Operasi Besar di Singapura
masuk
memasukkan berita id  1863166
Insert berita  Pemuda di Cirebon Hilang Terseret Arus saat Ikut Tradisi Berenang Sungai Cipager
masuk
memasukkan berita id  1863164
Insert berita  Dukun di Bali Tewas Dibacok Pasiennya Sendiri
masuk
memasukkan berita id  1863163
Insert berita  Ternyata Calon Orangtua Paling Sering <i>Googling</i> Nama Anak
masuk
memasukkan berita id  1863162
Insert berita  Pemerintah Buat Lapak Baru untuk IKM dan UKM Jualan <i>Online</i>
masuk
memasukkan berita id  1863161
Insert berita  Kemenpar Luncurkan Festival Pesona Tambor

masuk
page  24
https://index.okezone.com/bydate/index/2018/02/22/345/
memasukkan berita id  1863091
Insert berita  Belanja K/L Pakai Kartu Kredit Rp200 Juta, Jangan Sampai Salah Gesek
masuk
memasukkan berita id  1863085
Insert berita  YouTuber yang Memilih Menjadi Vegetarian untuk Mengobati Kanker Akhirnya Meninggal Dunia
masuk
memasukkan berita id  1863090
Insert berita  RI Persiapkan Energi Terbarukan, Menteri Jonan: Harga Harus Terjangkau
masuk
memasukkan berita id  1863089
Insert berita  TKI di Taiwan Tewas Misterius, Sempat Tulis Surat Wasiat dan Siaran <i>Live</i> di Facebook
masuk
memasukkan berita id  1863088
Insert berita  KPK-Polri Bentuk Tim Penghubung Informasi untuk Usut Kasus Novel Baswedan
masuk
memasukkan berita id  1863087
Insert berita  Bank Mandiri Targetkan Penyaluran KPR Naik 15% Jadi Rp45,6 Triliun
masuk
memasukkan berita id  1863086
Insert berita  Diizinkan Pulang, Novel Baswedan Dua Bulan Lagi Harus Kembali ke Singapura      
masuk
memasukkan berita id  1863084


KeyboardInterrupt: 